In [24]:
import pandas as pd
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import statistics 
import math
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression

In [ ]:
# Code from Alan Couzens, and modified for my use case 

In [37]:
# read in trainingPeaks data 
df = pd.read_csv('workout1.csv')


,Title,WorkoutType,WorkoutDescription,PlannedDuration,PlannedDistanceInMeters,WorkoutDay,CoachComments,DistanceInMeters,PowerAverage,PowerMax,...,PWRZone3Minutes,PWRZone4Minutes,PWRZone5Minutes,PWRZone6Minutes,PWRZone7Minutes,PWRZone8Minutes,PWRZone9Minutes,PWRZone10Minutes,Rpe,Feeling
0,Vo2 ints. 5x4 Dry,Run,NaN,0.75,8046.722656,6/1/2021,NaN,7413.0000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Training,Strength,NaN,0.50,NaN,6/1/2021,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
2,Zone 2 run,Run,NaN,1.00,11265.411130,6/2/2021,NaN,11394.0000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
3,Field hike,Walk,NaN,NaN,NaN,6/2/2021,NaN,10621.6704,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Baby baldy 2x10 hard climb,Run,NaN,1.75,7242.050293,6/3/2021,NaN,7329.0000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
runs = df.loc[df['WorkoutType'] == 'Run']
cols = ['Title','WorkoutType', 'WorkoutDescription','PlannedDuration','PlannedDistanceInMeters','WorkoutDay','CoachComments',
        'DistanceInMeters','AthleteComments','TimeTotalInHours','VelocityAverage','VelocityMax','CadenceAverage','CadenceMax',
        'HeartRateAverage','HeartRateMax','IF','TSS','HRZone1Minutes','HRZone2Minutes','HRZone3Minutes','HRZone4Minutes',
        'HRZone5Minutes','HRZone6Minutes','HRZone7Minutes', 'Rpe','Feeling']

#PaceZ1 = 

df = df[cols]
runs = runs[cols]
runs.head(5)

,Title,WorkoutType,WorkoutDescription,PlannedDuration,PlannedDistanceInMeters,WorkoutDay,CoachComments,DistanceInMeters,AthleteComments,TimeTotalInHours,...,TSS,HRZone1Minutes,HRZone2Minutes,HRZone3Minutes,HRZone4Minutes,HRZone5Minutes,HRZone6Minutes,HRZone7Minutes,Rpe,Feeling
0,Vo2 ints. 5x4 Dry,Run,NaN,0.750000,8046.722656,6/1/2021,NaN,7413.0,NaN,0.824861,...,39.59,9.0,28.0,5.0,4.0,5.0,2.0,0.0,NaN,NaN
2,Zone 2 run,Run,NaN,1.000000,11265.411130,6/2/2021,NaN,11394.0,NaN,1.182250,...,60.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
4,Baby baldy 2x10 hard climb,Run,NaN,1.750000,7242.050293,6/3/2021,NaN,7329.0,*06/03/2021 Christopher Skylar McDaniel: Milo...,1.109944,...,48.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BST-millville-MillvilleCanyon,Run,NaN,4.000000,25749.511720,6/5/2021,NaN,26349.0,NaN,3.396472,...,156.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Zone 2 run,Run,NaN,1.083333,11265.411130,6/7/2021,NaN,12156.0,NaN,1.204833,...,67.01,35.0,44.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [3]:
# Linear model
def get_block_end(start, block_length):
    end = start + timedelta(days=block_length)

In [4]:
def sum_TIZ_between_2_dates(data, col, date1, date2):
    date1 = pd.to_datetime(date1)
    date2 = pd.to_datetime(date2)
    total = data.loc[(data.WorkoutDay >= date1) & (data.WorkoutDay <= date2), col].sum()
    return total


In [17]:
# I would love to get this data in pace bands as well, although HR is more stable. I do have HR for most of these 
# I will use a regression on average pace later using just the run df

def get_input_data_from_TrainingPeaks(file, block_length_days):
    data = df
    data['WorkoutDay'] = pd.to_datetime(data['WorkoutDay'])
    start = data['WorkoutDay'].values[0]
    end = data['WorkoutDay'].values[-1]
    days = (end - start)/np.timedelta64(1,'D')
    blocks = int(days/block_length_days)
    block_dates = []
    for i in range(blocks+1):
        new_date = start + np.timedelta64(i*block_length_days, 'D')
        block_dates.append(np.datetime64(new_date, 'D'))
    intensities = ['HRZone1Minutes', 'HRZone2Minutes', 'HRZone3Minutes', 'HRZone4Minutes', 'HRZone5Minutes', 'HRZone6Minutes', 'HRZone7Minutes']
    input_data = []
    for i in range(len(block_dates)):
        try:
            block_TIZ = []
            for intensity in intensities:
                total_TIZ = sum_TIZ_between_2_dates(data, intensity, block_dates[i], block_dates[i+1])
                block_TIZ.append(total_TIZ)
            input_data.append(block_TIZ)
        except IndexError:
            pass
    return input_data



In [29]:
import numpy as np
time_in_zone_by_month = get_input_data_from_TrainingPeaks('workout1.csv', 24)
     
#Output variable - tested Threshold pace each month (replace with your own data)
ThreshPace_by_month = [420,420,400,400,395,385,385,395,385,380,375,370,385,385,385,375,367,365,363]

print(time_in_zone_by_month)

[[256.0, 820.0, 247.0, 59.0, 19.0, 25.0, 0.0], [917.0, 848.0, 339.0, 176.0, 50.0, 51.0, 0.0], [532.0, 790.0, 245.0, 92.0, 13.0, 5.0, 0.0], [557.0, 633.0, 349.0, 53.0, 8.0, 5.0, 0.0], [1332.0, 878.0, 343.0, 124.0, 12.0, 0.0, 0.0], [83.0, 261.0, 119.0, 53.0, 18.0, 6.0, 0.0], [267.0, 315.0, 286.0, 154.0, 44.0, 5.0, 0.0], [34.0, 84.0, 66.0, 28.0, 2.0, 1.0, 0.0], [23.0, 64.0, 22.0, 16.0, 21.0, 2.0, 0.0], [28.0, 133.0, 48.0, 23.0, 1.0, 0.0, 0.0], [494.0, 339.0, 120.0, 69.0, 7.0, 0.0, 0.0], [148.0, 586.0, 215.0, 60.0, 7.0, 1.0, 0.0], [200.0, 690.0, 305.0, 93.0, 10.0, 3.0, 0.0], [248.0, 769.0, 378.0, 126.0, 39.0, 13.0, 1.0], [177.0, 851.0, 380.0, 84.0, 44.0, 6.0, 0.0], [85.0, 476.0, 224.0, 50.0, 4.0, 0.0, 0.0], [292.0, 770.0, 446.0, 178.0, 32.0, 12.0, 0.0], [174.0, 891.0, 593.0, 184.0, 47.0, 22.0, 0.0], [714.0, 733.0, 572.0, 88.0, 24.0, 15.0, 2.0]]


In [1]:

X = time_in_zone_by_month
y = ThreshPace_by_month

model = LinearRegression().fit(X,y)
"""Print out model weights for each zone so that you can see how strongly they are
weighted/how much each minute in each zone is worth. e.g. the first coefficient is the
weight for Zone 1, the second is the weight for Zone 2 etc."""
print(model.coef_)

print(model.intercept_)
    


NameError: name 'time_in_zone_by_month' is not defined